In [1]:

%pprint

Pretty printing has been turned OFF


In [2]:

%%time
from neo4j import GraphDatabase
%run ../load_magic/storage.py
%run ../load_magic/dataframes.py

s = Storage()
uri = 'bolt://localhost:7687'
user = 'neo4j'
password = 'Genesis11'
driver = GraphDatabase.driver(uri, auth=(user, password))

Wall time: 3.64 s


In [3]:

import os

def clear_database(tx):
    delete_str = "CALL apoc.periodic.iterate('MATCH (n) RETURN n', 'DETACH DELETE n', {batchSize:1000});"
    tx.run(delete_str)
def create_constraint(tx, class_name):
    constraint_str = 'CREATE CONSTRAINT Unique{} IF NOT EXISTS ON (x:{}) ASSERT x.id IS UNIQUE;'
    constraint_str = constraint_str.format(class_name, class_name)
    tx.run(constraint_str)
def f(x):
    if type(x) is str:
        x = x.strip()
    
    return x
dbmss_dir = r'C:\Users\dev\.Neo4jDesktop\relate-data\dbmss'
import_dir = os.path.join(dbmss_dir, 'dbms-2f3851f5-6aa7-490d-9a58-751b8b51f3ab', 'import')
csv_dir = '../saves/csv'

In [4]:

import pandas as pd

def do_cypher_tx(tx, cypher):
    results_list = tx.run(cypher)
    values_list = []
    for record in results_list:
        values_list.append(dict(record.items()))
    
    return values_list

In [6]:

tx_str = "CALL dbms.procedures() YIELD name, signature RETURN name, signature"
with driver.session() as session:
    rows_list = session.read_transaction(do_cypher_tx, tx_str)
sorted(pd.DataFrame(rows_list).name.tolist())

['db.awaitIndex', 'db.awaitIndexes', 'db.clearQueryCaches', 'db.constraints', 'db.createIndex', 'db.createLabel', 'db.createProperty', 'db.createRelationshipType', 'db.createUniquePropertyConstraint', 'db.index.fulltext.awaitEventuallyConsistentIndexRefresh', 'db.index.fulltext.createNodeIndex', 'db.index.fulltext.createRelationshipIndex', 'db.index.fulltext.drop', 'db.index.fulltext.listAvailableAnalyzers', 'db.index.fulltext.queryNodes', 'db.index.fulltext.queryRelationships', 'db.indexDetails', 'db.indexes', 'db.info', 'db.labels', 'db.ping', 'db.prepareForReplanning', 'db.propertyKeys', 'db.relationshipTypes', 'db.resampleIndex', 'db.resampleOutdatedIndexes', 'db.schema.nodeTypeProperties', 'db.schema.relTypeProperties', 'db.schema.visualization', 'db.schemaStatements', 'db.stats.clear', 'db.stats.collect', 'db.stats.retrieve', 'db.stats.retrieveAllAnonymized', 'db.stats.status', 'db.stats.stop', 'dbms.cluster.routing.getRoutingTable', 'dbms.components', 'dbms.database.state', 'dbm

In [7]:

from neo4jrestclient.client import GraphDatabase

[f'GraphDatabase.{fn}' for fn in dir(GraphDatabase) if not fn.startswith('_')]

['GraphDatabase.extensions', 'GraphDatabase.flush', 'GraphDatabase.labels', 'GraphDatabase.query', 'GraphDatabase.reference_node', 'GraphDatabase.transaction', 'GraphDatabase.traversal', 'GraphDatabase.traverse']

In [8]:

try:
    gdb = GraphDatabase('http://localhost:7474/', username=user, password=password)
    print([f'gdb.{fn}' for fn in dir(gdb) if not fn.startswith('_')])
except Exception as e:
    print(str(e).strip())

'relationship_index'


In [9]:

with driver.session() as session:
    try:
        session.write_transaction(clear_database)
    except Exception as e:
        print(str(e).strip())

{code: Neo.ClientError.Procedure.ProcedureNotFound} {message: There is no procedure with the name `apoc.periodic.iterate` registered for this database instance. Please ensure you've spelled the procedure name correctly and that the procedure is properly deployed.}


In [16]:

import os
from shutil import copyfile

import_dir = r'C:/neo4j/import'
for sub_directory, directories_list, files_list in os.walk(csv_dir):
    for csv_name in files_list:
        if csv_name.endswith('.csv') and (csv_name[0] == csv_name[0].upper()):
            class_name = csv_name.split('.')[0]
            with driver.session() as session:
                session.write_transaction(create_constraint, class_name)
            src = os.path.abspath(os.path.join(sub_directory, csv_name))
            dst = os.path.abspath(os.path.join(import_dir, csv_name))
            copyfile(src, dst)


## Set Primary Keys

In [17]:

file_names_str = '''LOAD CSV WITH HEADERS FROM 'file:///FileNames.csv' AS row
WITH
    toInteger(row.file_name_id) AS file_name_id,
    row.file_name AS file_name,
    row.percent_fit AS percent_fit,
    row.is_opportunity_application_emailed AS is_opportunity_application_emailed,
    row.opportunity_application_email_date AS opportunity_application_email_date,
    row.is_remote_delivery AS is_remote_delivery,
    row.manager_notes AS manager_notes
MERGE (x:FileNames {file_name_id: file_name_id}) SET
    x.file_name = file_name,
    x.percent_fit = percent_fit,
    x.is_opportunity_application_emailed = is_opportunity_application_emailed,
    x.opportunity_application_email_date = opportunity_application_email_date,
    x.is_remote_delivery = is_remote_delivery,
    x.manager_notes = manager_notes;'''
with driver.session() as session:
    session.write_transaction(do_cypher_tx, file_names_str)

In [18]:

header_tags_str = '''LOAD CSV WITH HEADERS FROM 'file:///HeaderTags.csv' AS row
WITH
    toInteger(row.header_tag_id) AS header_tag_id,
    row.is_in_document_structure_elements_set AS is_in_document_structure_elements_set,
    row.is_in_document_head_elements_set AS is_in_document_head_elements_set,
    row.is_in_document_body_elements_set AS is_in_document_body_elements_set,
    row.is_in_block_elements_set AS is_in_block_elements_set,
    row.is_in_basic_text_set AS is_in_basic_text_set,
    row.is_in_section_headings_set AS is_in_section_headings_set,
    row.is_in_lists_set AS is_in_lists_set,
    row.is_in_other_block_elements_set AS is_in_other_block_elements_set,
    row.is_in_inline_elements_set AS is_in_inline_elements_set,
    row.is_in_anchor_set AS is_in_anchor_set,
    row.is_in_phrase_elements_set AS is_in_phrase_elements_set,
    row.is_in_general_set AS is_in_general_set,
    row.is_in_computer_phrase_elements_set AS is_in_computer_phrase_elements_set,
    row.is_in_presentation_set AS is_in_presentation_set,
    row.is_in_span_set AS is_in_span_set,
    row.is_in_other_inline_elements_set AS is_in_other_inline_elements_set,
    row.is_in_images_and_objects_set AS is_in_images_and_objects_set,
    row.is_in_forms_set AS is_in_forms_set,
    row.is_in_tables_set AS is_in_tables_set,
    row.is_in_frames_set AS is_in_frames_set,
    row.is_in_historic_elements_set AS is_in_historic_elements_set,
    row.is_in_non_standard_elements_set AS is_in_non_standard_elements_set,
    row.header_tag AS header_tag
MERGE (x:HeaderTags {header_tag_id: header_tag_id}) SET
    x.is_in_document_structure_elements_set = is_in_document_structure_elements_set,
    x.is_in_document_head_elements_set = is_in_document_head_elements_set,
    x.is_in_document_body_elements_set = is_in_document_body_elements_set,
    x.is_in_block_elements_set = is_in_block_elements_set,
    x.is_in_basic_text_set = is_in_basic_text_set,
    x.is_in_section_headings_set = is_in_section_headings_set,
    x.is_in_lists_set = is_in_lists_set,
    x.is_in_other_block_elements_set = is_in_other_block_elements_set,
    x.is_in_inline_elements_set = is_in_inline_elements_set,
    x.is_in_anchor_set = is_in_anchor_set,
    x.is_in_phrase_elements_set = is_in_phrase_elements_set,
    x.is_in_general_set = is_in_general_set,
    x.is_in_computer_phrase_elements_set = is_in_computer_phrase_elements_set,
    x.is_in_presentation_set = is_in_presentation_set,
    x.is_in_span_set = is_in_span_set,
    x.is_in_other_inline_elements_set = is_in_other_inline_elements_set,
    x.is_in_images_and_objects_set = is_in_images_and_objects_set,
    x.is_in_forms_set = is_in_forms_set,
    x.is_in_tables_set = is_in_tables_set,
    x.is_in_frames_set = is_in_frames_set,
    x.is_in_historic_elements_set = is_in_historic_elements_set,
    x.is_in_non_standard_elements_set = is_in_non_standard_elements_set,
    x.header_tag = header_tag;'''
with driver.session() as session:
    session.write_transaction(do_cypher_tx, header_tags_str)

In [19]:

header_tag_sequence_str = '''LOAD CSV WITH HEADERS FROM 'file:///HeaderTagSequence.csv' AS row
WITH
    toInteger(row.header_tag_sequence_id) AS header_tag_sequence_id,
    toInteger(row.file_name_id) AS file_name_id,
    toInteger(row.header_tag_id) AS header_tag_id,
    toInteger(row.sequence_order) AS sequence_order
MERGE (x:HeaderTagSequence {header_tag_sequence_id: header_tag_sequence_id}) SET
    x.file_name_id = file_name_id,
    x.header_tag_id = header_tag_id,
    x.sequence_order = sequence_order;'''
with driver.session() as session:
    session.write_transaction(do_cypher_tx, header_tag_sequence_str)

In [20]:

minimum_requirements_section_str = '''LOAD CSV WITH HEADERS FROM 'file:///MinimumRequirementsSection.csv' AS row
WITH
    toInteger(row.mrs_id) AS mrs_id,
    row.mrs_symbol AS mrs_symbol,
    row.mrs_explanation AS mrs_explanation
MERGE (x:MinimumRequirementsSection {mrs_id: mrs_id}) SET
    x.mrs_symbol = mrs_symbol,
    x.mrs_explanation = mrs_explanation;'''
with driver.session() as session:
    session.write_transaction(do_cypher_tx, minimum_requirements_section_str)

In [21]:

navigable_parents_str = '''LOAD CSV WITH HEADERS FROM 'file:///NavigableParents.csv' AS row
WITH
    toInteger(row.navigable_parent_id) AS navigable_parent_id,
    toInteger(row.header_tag_id) AS header_tag_id,
    row.navigable_parent AS navigable_parent,
    row.is_header AS is_header,
    row.is_task_scope AS is_task_scope,
    row.is_minimum_qualification AS is_minimum_qualification,
    row.is_preferred_qualification AS is_preferred_qualification,
    row.is_legal_notification AS is_legal_notification,
    row.is_job_title AS is_job_title,
    row.is_office_location AS is_office_location,
    row.is_job_duration AS is_job_duration,
    row.is_supplemental_pay AS is_supplemental_pay,
    row.is_educational_requirement AS is_educational_requirement,
    row.is_interview_procedure AS is_interview_procedure,
    row.is_corporate_scope AS is_corporate_scope,
    row.is_posting_date AS is_posting_date,
    row.is_other AS is_other,
    row.is_qualification AS is_qualification
MERGE (x:NavigableParents {navigable_parent_id: navigable_parent_id}) SET
    x.header_tag_id = header_tag_id,
    x.navigable_parent = navigable_parent,
    x.is_header = is_header,
    x.is_task_scope = is_task_scope,
    x.is_minimum_qualification = is_minimum_qualification,
    x.is_preferred_qualification = is_preferred_qualification,
    x.is_legal_notification = is_legal_notification,
    x.is_job_title = is_job_title,
    x.is_office_location = is_office_location,
    x.is_job_duration = is_job_duration,
    x.is_supplemental_pay = is_supplemental_pay,
    x.is_educational_requirement = is_educational_requirement,
    x.is_interview_procedure = is_interview_procedure,
    x.is_corporate_scope = is_corporate_scope,
    x.is_posting_date = is_posting_date,
    x.is_other = is_other,
    x.is_qualification = is_qualification;'''
with driver.session() as session:
    session.write_transaction(do_cypher_tx, navigable_parents_str)

In [23]:

nvaigable_parent_sequence_str = '''LOAD CSV WITH HEADERS FROM 'file:///NavigableParentSequence.csv' AS row
WITH
    toInteger(row.navigable_parent_sequence_id) AS navigable_parent_sequence_id,
    toInteger(row.file_name_id) AS file_name_id,
    toInteger(row.navigable_parent_id) AS navigable_parent_id,
    toInteger(row.sequence_order) AS sequence_order,
    toInteger(row.mrs_id) AS mrs_id
MERGE (x:NavigableParentSequence {navigable_parent_sequence_id: navigable_parent_sequence_id}) SET
    x.file_name_id = file_name_id,
    x.navigable_parent_id = navigable_parent_id,
    x.sequence_order = sequence_order,
    x.mrs_id = mrs_id;'''
with driver.session() as session:
    session.write_transaction(do_cypher_tx, nvaigable_parent_sequence_str)

In [24]:

parts_of_speech_str = '''LOAD CSV WITH HEADERS FROM 'file:///PartsOfSpeech.csv' AS row
WITH
    toInteger(row.pos_id) AS pos_id,
    row.is_header AS is_header,
    row.is_task_scope AS is_task_scope,
    row.is_minimum_qualification AS is_minimum_qualification,
    row.is_preferred_qualification AS is_preferred_qualification,
    row.is_legal_notification AS is_legal_notification,
    row.is_job_title AS is_job_title,
    row.is_office_location AS is_office_location,
    row.is_job_duration AS is_job_duration,
    row.is_supplemental_pay AS is_supplemental_pay,
    row.is_educational_requirement AS is_educational_requirement,
    row.is_interview_procedure AS is_interview_procedure,
    row.is_corporate_scope AS is_corporate_scope,
    row.is_posting_date AS is_posting_date,
    row.is_other AS is_other,
    row.pos_symbol AS pos_symbol,
    row.pos_explanation AS pos_explanation
MERGE (x:PartsOfSpeech {pos_id: pos_id}) SET
    x.is_header = is_header,
    x.is_task_scope = is_task_scope,
    x.is_minimum_qualification = is_minimum_qualification,
    x.is_preferred_qualification = is_preferred_qualification,
    x.is_legal_notification = is_legal_notification,
    x.is_job_title = is_job_title,
    x.is_office_location = is_office_location,
    x.is_job_duration = is_job_duration,
    x.is_supplemental_pay = is_supplemental_pay,
    x.is_educational_requirement = is_educational_requirement,
    x.is_interview_procedure = is_interview_procedure,
    x.is_corporate_scope = is_corporate_scope,
    x.is_posting_date = is_posting_date,
    x.is_other = is_other,
    x.pos_symbol = pos_symbol,
    x.pos_explanation = pos_explanation;'''
with driver.session() as session:
    session.write_transaction(do_cypher_tx, parts_of_speech_str)


## Set Foreign Keys

In [25]:

tx_str = """
MATCH (a:FileNames), (b:HeaderTagSequence) 
   WHERE a.file_name_id = b.file_name_id
CREATE (b)-[r: IS_CONTAINED_IN]->(a);"""
with driver.session() as session:
    session.write_transaction(do_cypher_tx, tx_str)

In [26]:

tx_str = """
MATCH (a:FileNames), (b:NavigableParentSequence) 
   WHERE a.file_name_id = b.file_name_id
CREATE (b)-[r: IS_CONTAINED_IN]->(a);"""
with driver.session() as session:
    session.write_transaction(do_cypher_tx, tx_str)

In [27]:

tx_str = """
MATCH (a:HeaderTags), (b:HeaderTagSequence) 
   WHERE a.header_tag_id = b.header_tag_id
CREATE (a)-[r: IS_PART_OF]->(b);"""
with driver.session() as session:
    session.write_transaction(do_cypher_tx, tx_str)

In [28]:

tx_str = """
MATCH (a:HeaderTags), (b:NavigableParents) 
   WHERE a.header_tag_id = b.header_tag_id
CREATE (a)-[r: SUMMARIZES]->(b);"""
with driver.session() as session:
    session.write_transaction(do_cypher_tx, tx_str)

In [29]:

tx_str = """
MATCH (a:MinimumRequirementsSection), (b:NavigableParentSequence) 
   WHERE a.mrs_id = b.mrs_id
CREATE (a)-[r: SUMMARIZES]->(b);"""
with driver.session() as session:
    session.write_transaction(do_cypher_tx, tx_str)

In [30]:

tx_str = """
MATCH (a:NavigableParents), (b:NavigableParentSequence) 
   WHERE a.navigable_parent_id = b.navigable_parent_id
CREATE (a)-[r: IS_PART_OF]->(b);"""
with driver.session() as session:
    session.write_transaction(do_cypher_tx, tx_str)

In [5]:

for a in ['True', 'False']:
    for b in ['is_task_scope', 'is_minimum_qualification', 'is_preferred_qualification', 'is_legal_notification', 'is_job_title', 'is_office_location',
              'is_job_duration', 'is_supplemental_pay', 'is_educational_requirement', 'is_interview_procedure', 'is_corporate_scope', 'is_posting_date', 'is_other']:
        tx_str = f"""
            MATCH (pos:PartsOfSpeech {{is_header: '{a}', {b}: 'True'}}), (np:NavigableParents {{is_header: '{a}', {b}: 'True'}})
            CREATE (pos)-[r: SUMMARIZES]->(np);"""
        with driver.session() as session:
#             print(tx_str)
            session.write_transaction(do_cypher_tx, tx_str)


----
## Set Attributes from Pickles

In [35]:

# Set attributes from pickles
import os

pickle_dir = '../saves/pickle'
files_list = os.listdir(pickle_dir)
for file_name in files_list:
    if file_name.startswith('NAVIGABLE_PARENT_IS_') and file_name.endswith('_NONHEADER_DICT.pickle'):
        pickle_name = file_name.split('.')[0]
        attribute_name = '_'.join(pickle_name.lower().split('_')[2:-2])
        nonheader_dict = s.load_object(pickle_name)
        for navigable_parent, boolean_value in nonheader_dict.items():
            tx_str = f"""MATCH (np:NavigableParents {{navigable_parent: '{navigable_parent.replace("'", chr(92) + "'")}'}})
SET np.{attribute_name} = '{boolean_value}'"""
            with driver.session() as session:
                session.write_transaction(do_cypher_tx, tx_str)

In [39]:

BASIC_TAGS_DICT = s.load_object('BASIC_TAGS_DICT')
for navigable_parent, is_header in BASIC_TAGS_DICT.items():
    tx_str = f"""MATCH (np:NavigableParents {{navigable_parent: '{navigable_parent.replace("'", chr(92) + "'")}'}})
SET np.is_header = '{is_header}'"""
    with driver.session() as session:
        session.write_transaction(do_cypher_tx, tx_str)

In [42]:

corp_scope_headers_list = s.load_object('corp_scope_headers_list')
for navigable_parent in corp_scope_headers_list:
    tx_str = f"""MATCH (np:NavigableParents {{navigable_parent: '{navigable_parent.replace("'", chr(92) + "'")}'}})
SET np.is_corporate_scope = 'True', np.is_header = 'True';"""
    with driver.session() as session:
        session.write_transaction(do_cypher_tx, tx_str)

In [43]:

CORP_SCOPE_NONHEADERS_LIST = s.load_object('CORP_SCOPE_NONHEADERS_LIST')
for navigable_parent in CORP_SCOPE_NONHEADERS_LIST:
    tx_str = f"""MATCH (np:NavigableParents {{navigable_parent: '{navigable_parent.replace("'", chr(92) + "'")}'}})
SET np.is_corporate_scope = 'True', np.is_header = 'False';"""
    with driver.session() as session:
        session.write_transaction(do_cypher_tx, tx_str)

In [50]:

educ_reqs_headers_list = s.load_object('educ_reqs_headers_list')
for navigable_parent in educ_reqs_headers_list:
    tx_str = f"""MATCH (np:NavigableParents {{navigable_parent: '{navigable_parent.replace("'", chr(92) + "'")}'}})
SET np.is_educational_requirement = 'True', np.is_header = 'True';"""
    with driver.session() as session:
        session.write_transaction(do_cypher_tx, tx_str)

In [60]:

H_RQ_DICT = s.load_object('H_RQ_DICT')
for navigable_parent, is_hrq in H_RQ_DICT.items():
    if is_hrq:
        tx_str = f"""MATCH (np:NavigableParents {{navigable_parent: '{navigable_parent.replace("'", chr(92) + "'")}'}})
SET np.is_minimum_qualification = 'True',  np.is_header = 'True';"""
        with driver.session() as session:
            session.write_transaction(do_cypher_tx, tx_str)
        tx_str = f"""MATCH
    (pos:PartsOfSpeech {{is_header: 'True', is_minimum_qualification: 'True'}}),
    (np:NavigableParents {{navigable_parent: '{navigable_parent.replace("'", chr(92) + "'")}'}})
MERGE (pos)-[r: SUMMARIZES]->(np);"""

In [49]:

interv_proc_headers_list = s.load_object('interv_proc_headers_list')
for navigable_parent in interv_proc_headers_list:
    tx_str = f"""MATCH (np:NavigableParents {{navigable_parent: '{navigable_parent.replace("'", chr(92) + "'")}'}})
SET np.is_interview_procedure = 'True', np.is_header = 'True';"""
    with driver.session() as session:
        session.write_transaction(do_cypher_tx, tx_str)

In [51]:

job_duration_headers_list = s.load_object('job_duration_headers_list')
for navigable_parent in job_duration_headers_list:
    tx_str = f"""MATCH (np:NavigableParents {{navigable_parent: '{navigable_parent.replace("'", chr(92) + "'")}'}})
SET np.is_job_duration = 'True', np.is_header = 'True';"""
    with driver.session() as session:
        session.write_transaction(do_cypher_tx, tx_str)

In [52]:

job_title_headers_list = s.load_object('job_title_headers_list')
for navigable_parent in job_title_headers_list:
    tx_str = f"""MATCH (np:NavigableParents {{navigable_parent: '{navigable_parent.replace("'", chr(92) + "'")}'}})
SET np.is_job_title = 'True', np.is_header = 'True';"""
    with driver.session() as session:
        session.write_transaction(do_cypher_tx, tx_str)

In [53]:

legal_notifs_headers_list = s.load_object('legal_notifs_headers_list')
for navigable_parent in legal_notifs_headers_list:
    tx_str = f"""MATCH (np:NavigableParents {{navigable_parent: '{navigable_parent.replace("'", chr(92) + "'")}'}})
SET np.is_legal_notification = 'True', np.is_header = 'True';"""
    with driver.session() as session:
        session.write_transaction(do_cypher_tx, tx_str)

In [54]:

LEGAL_NOTIFS_NONHEADERS_LIST = s.load_object('LEGAL_NOTIFS_NONHEADERS_LIST')
for navigable_parent in LEGAL_NOTIFS_NONHEADERS_LIST:
    tx_str = f"""MATCH (np:NavigableParents {{navigable_parent: '{navigable_parent.replace("'", chr(92) + "'")}'}})
SET np.is_legal_notification = 'True', np.is_header = 'False';"""
    with driver.session() as session:
        session.write_transaction(do_cypher_tx, tx_str)

In [55]:

NAVIGABLE_PARENT_IS_HEADER_DICT = s.load_object('NAVIGABLE_PARENT_IS_HEADER_DICT')
for navigable_parent, is_header in NAVIGABLE_PARENT_IS_HEADER_DICT.items():
    tx_str = f"""MATCH (np:NavigableParents {{navigable_parent: '{navigable_parent.replace("'", chr(92) + "'")}'}})
SET np.is_header = '{is_header}'"""
    with driver.session() as session:
        session.write_transaction(do_cypher_tx, tx_str)

In [56]:

office_loc_headers_list = s.load_object('office_loc_headers_list')
for navigable_parent in office_loc_headers_list:
    tx_str = f"""MATCH (np:NavigableParents {{navigable_parent: '{navigable_parent.replace("'", chr(92) + "'")}'}})
SET np.is_office_location = 'True', np.is_header = 'True';"""
    with driver.session() as session:
        session.write_transaction(do_cypher_tx, tx_str)

In [57]:

OFFICE_LOC_NONHEADERS_LIST = s.load_object('OFFICE_LOC_NONHEADERS_LIST')
for navigable_parent in OFFICE_LOC_NONHEADERS_LIST:
    tx_str = f"""MATCH (np:NavigableParents {{navigable_parent: '{navigable_parent.replace("'", chr(92) + "'")}'}})
SET np.is_office_location = 'True', np.is_header = 'False';"""
    with driver.session() as session:
        session.write_transaction(do_cypher_tx, tx_str)

In [58]:

other_headers_list = s.load_object('other_headers_list')
for navigable_parent in other_headers_list:
    tx_str = f"""MATCH (np:NavigableParents {{navigable_parent: '{navigable_parent.replace("'", chr(92) + "'")}'}})
SET np.is_other = 'True', np.is_header = 'True';"""
    with driver.session() as session:
        session.write_transaction(do_cypher_tx, tx_str)

In [61]:

O_RQ_DICT = s.load_object('O_RQ_DICT')
for navigable_parent, is_orq in O_RQ_DICT.items():
    if is_orq:
        tx_str = f"""MATCH (np:NavigableParents {{navigable_parent: '{navigable_parent.replace("'", chr(92) + "'")}'}})
SET np.is_minimum_qualification = 'True',  np.is_header = 'False';"""
        with driver.session() as session:
            session.write_transaction(do_cypher_tx, tx_str)
        tx_str = f"""MATCH
    (pos:PartsOfSpeech {{is_header: 'False', is_minimum_qualification: 'True'}}),
    (np:NavigableParents {{navigable_parent: '{navigable_parent.replace("'", chr(92) + "'")}'}})
MERGE (pos)-[r: SUMMARIZES]->(np);"""

In [62]:

O_TS_DICT = s.load_object('O_TS_DICT')
for navigable_parent, is_orq in O_TS_DICT.items():
    if is_orq:
        tx_str = f"""MATCH (np:NavigableParents {{navigable_parent: '{navigable_parent.replace("'", chr(92) + "'")}'}})
SET np.is_task_scope = 'True',  np.is_header = 'False';"""
        with driver.session() as session:
            session.write_transaction(do_cypher_tx, tx_str)
        tx_str = f"""MATCH
    (pos:PartsOfSpeech {{is_header: 'False', is_task_scope: 'True'}}),
    (np:NavigableParents {{navigable_parent: '{navigable_parent.replace("'", chr(92) + "'")}'}})
MERGE (pos)-[r: SUMMARIZES]->(np);"""

In [63]:

post_date_headers_list = s.load_object('post_date_headers_list')
for navigable_parent in post_date_headers_list:
    tx_str = f"""MATCH (np:NavigableParents {{navigable_parent: '{navigable_parent.replace("'", chr(92) + "'")}'}})
SET np.is_posting_date = 'True', np.is_header = 'True';"""
    with driver.session() as session:
        session.write_transaction(do_cypher_tx, tx_str)

In [64]:

preff_quals_headers_list = s.load_object('preff_quals_headers_list')
for navigable_parent in preff_quals_headers_list:
    tx_str = f"""MATCH (np:NavigableParents {{navigable_parent: '{navigable_parent.replace("'", chr(92) + "'")}'}})
SET np.is_preferred_qualification = 'True', np.is_header = 'True';"""
    with driver.session() as session:
        session.write_transaction(do_cypher_tx, tx_str)

In [65]:

PREFF_QUALS_NONHEADERS_LIST = s.load_object('PREFF_QUALS_NONHEADERS_LIST')
for navigable_parent in PREFF_QUALS_NONHEADERS_LIST:
    tx_str = f"""MATCH (np:NavigableParents {{navigable_parent: '{navigable_parent.replace("'", chr(92) + "'")}'}})
SET np.is_preferred_qualification = 'True', np.is_header = 'False';"""
    with driver.session() as session:
        session.write_transaction(do_cypher_tx, tx_str)

In [68]:

req_quals_headers_list = s.load_object('req_quals_headers_list')
for navigable_parent in req_quals_headers_list:
    tx_str = f"""MATCH (np:NavigableParents {{navigable_parent: '{navigable_parent.replace("'", chr(92) + "'")}'}})
SET np.is_minimum_qualification = 'True', np.is_header = 'True';"""
    with driver.session() as session:
        session.write_transaction(do_cypher_tx, tx_str)

In [69]:

supp_pay_headers_list = s.load_object('supp_pay_headers_list')
for navigable_parent in supp_pay_headers_list:
    tx_str = f"""MATCH (np:NavigableParents {{navigable_parent: '{navigable_parent.replace("'", chr(92) + "'")}'}})
SET np.is_supplemental_pay = 'True', np.is_header = 'True';"""
    with driver.session() as session:
        session.write_transaction(do_cypher_tx, tx_str)

In [70]:

supp_pay_nonheaders_list = s.load_object('supp_pay_nonheaders_list')
for navigable_parent in supp_pay_nonheaders_list:
    tx_str = f"""MATCH (np:NavigableParents {{navigable_parent: '{navigable_parent.replace("'", chr(92) + "'")}'}})
SET np.is_supplemental_pay = 'True', np.is_header = 'False';"""
    with driver.session() as session:
        session.write_transaction(do_cypher_tx, tx_str)

In [71]:

task_scope_headers_list = s.load_object('task_scope_headers_list')
for navigable_parent in task_scope_headers_list:
    tx_str = f"""MATCH (np:NavigableParents {{navigable_parent: '{navigable_parent.replace("'", chr(92) + "'")}'}})
SET np.is_task_scope = 'True', np.is_header = 'True';"""
    with driver.session() as session:
        session.write_transaction(do_cypher_tx, tx_str)

In [72]:

TASK_SCOPE_NONHEADERS_LIST = s.load_object('TASK_SCOPE_NONHEADERS_LIST')
for navigable_parent in TASK_SCOPE_NONHEADERS_LIST:
    tx_str = f"""MATCH (np:NavigableParents {{navigable_parent: '{navigable_parent.replace("'", chr(92) + "'")}'}})
SET np.is_task_scope = 'True', np.is_header = 'False';"""
    with driver.session() as session:
        session.write_transaction(do_cypher_tx, tx_str)

In [8]:

POS_DICT = s.load_object('POS_DICT')
for navigable_parent, pos_symbol in POS_DICT.items():
    tx_str = f"""MATCH
    (pos:PartsOfSpeech {{pos_symbol: '{pos_symbol}'}}),
    (np:NavigableParents {{navigable_parent: '{navigable_parent.replace("'", chr(92) + "'")}'}})
MERGE (pos)-[r: SUMMARIZES]->(np);"""
    with driver.session() as session:
        session.write_transaction(do_cypher_tx, tx_str)

In [36]:

for file_name in files_list:
    print(file_name)

.ipynb_checkpoints
basic_quals_clf.pickle
basic_quals_df.pickle
basic_tags_clf.pickle
basic_tags_df.pickle
BASIC_TAGS_DICT.pickle
bq_cv_vocab.pickle
bq_tt.pickle
CHILDLESS_TAGS_LIST.pickle
CHILD_STRS_LIST_DICT.pickle
CHILD_STR_CLF.pickle
child_str_df.pickle
corp_scope_headers_list.pickle
CORP_SCOPE_NONHEADERS_LIST.pickle
CRF.pickle
CS_CV.pickle
cs_cv_vocab.pickle
CS_TT.pickle
description_dict.pickle
educ_reqs_headers_list.pickle
entropy_df.pickle
entropy_tuples_list.pickle
estimators_list.pickle
FIT_ESTIMATORS_DICT.pickle
fit_estimators_list.pickle
GLOVE_BIG.pickle
GLOVE_SMALL.pickle
GN300_MODEL.pickle
HEADER_PATTERN_DICT.pickle
hunting_df.pickle
H_RQ_DICT.pickle
inference_durations_list.pickle
interv_proc_headers_list.pickle
job_duration_headers_list.pickle
job_title_headers_list.pickle
job_urls_list.pickle
LDA.pickle
LDA_CORPUS.pickle
LDA_DICTIONARY.pickle
legal_notifs_headers_list.pickle
LEGAL_NOTIFS_NONHEADERS_LIST.pickle
metrics_list.pickle
NAVIGABLE_PARENT_IS_CORPORATE_SCOPE_NONH


----
## Display Nodes and Relationships

In [9]:

print('CALL {')
union_list = []
for pos_symbol in ['H-TS', 'O-TS', 'H-RQ', 'O-RQ', 'H-PQ', 'O-PQ', 'H-LN', 'O-LN', 'H-JT', 'O-JT', 'H-OL', 'O-OL', 'H-JD', 'O-JD', 'H-SP', 'O-SP', 'H-ER', 'O-ER', 'H-IP',
            'O-IP', 'H-CS', 'O-CS', 'H-PD', 'O-PD', 'H-O', 'O-O']:
    cypher_str = """  MATCH (pos:PartsOfSpeech {pos_symbol: '"""
    cypher_str += pos_symbol
    cypher_str += """'})-->(np:NavigableParents)
  RETURN pos, np
  LIMIT 1"""
    union_list.append(cypher_str)
print("""
UNION
""".join(union_list))
print("""}
RETURN pos, np;""")

CALL {
  MATCH (pos:PartsOfSpeech {pos_symbol: 'H-TS'})-->(np:NavigableParents)
  RETURN pos, np
  LIMIT 1
UNION
  MATCH (pos:PartsOfSpeech {pos_symbol: 'O-TS'})-->(np:NavigableParents)
  RETURN pos, np
  LIMIT 1
UNION
  MATCH (pos:PartsOfSpeech {pos_symbol: 'H-RQ'})-->(np:NavigableParents)
  RETURN pos, np
  LIMIT 1
UNION
  MATCH (pos:PartsOfSpeech {pos_symbol: 'O-RQ'})-->(np:NavigableParents)
  RETURN pos, np
  LIMIT 1
UNION
  MATCH (pos:PartsOfSpeech {pos_symbol: 'H-PQ'})-->(np:NavigableParents)
  RETURN pos, np
  LIMIT 1
UNION
  MATCH (pos:PartsOfSpeech {pos_symbol: 'O-PQ'})-->(np:NavigableParents)
  RETURN pos, np
  LIMIT 1
UNION
  MATCH (pos:PartsOfSpeech {pos_symbol: 'H-LN'})-->(np:NavigableParents)
  RETURN pos, np
  LIMIT 1
UNION
  MATCH (pos:PartsOfSpeech {pos_symbol: 'O-LN'})-->(np:NavigableParents)
  RETURN pos, np
  LIMIT 1
UNION
  MATCH (pos:PartsOfSpeech {pos_symbol: 'H-JT'})-->(np:NavigableParents)
  RETURN pos, np
  LIMIT 1
UNION
  MATCH (pos:PartsOfSpeech {pos_symbol:

In [ ]:

TX_STR = '''
CALL {
  MATCH (a)-[r: IS_CONTAINED_IN]->(b)
  RETURN a, b
  ORDER BY rand()
  LIMIT 15
UNION
  MATCH (a)-[r: IS_PART_OF]->(b)
  RETURN a, b
  ORDER BY rand()
  LIMIT 15
UNION
  MATCH (a)-[r: SUMMARIZES]->(b)
  RETURN a, b
  ORDER BY rand()
  LIMIT 15
}
RETURN a, b;'''
def display_nodes_tx(tx):
    result = tx.run(TX_STR)
    print([f'result.{fn}' for fn in dir(result) if not fn.startswith('_')])
    record_dict = result.single()
    
    return record_dict
session = driver.session()
session.read_transaction(display_nodes_tx)

In [ ]:

TX_STR = '''
CALL {
  MATCH (x:FileNames)
  RETURN x
  LIMIT 10
UNION
  MATCH (x:HeaderTags)
  RETURN x
  LIMIT 10
UNION
  MATCH (x:HeaderTagSequence)
  RETURN x
  LIMIT 10
UNION
  MATCH (x:MinimumRequirementsSection)
  RETURN x
  LIMIT 10
UNION
  MATCH (x:NavigableParents)
  RETURN x
  LIMIT 10
UNION
  MATCH (x:NavigableParentSequence)
  RETURN x
  LIMIT 10
UNION
  MATCH (x:PartsOfSpeech)
  RETURN x
  LIMIT 10
}
RETURN x;'''
def display_nodes_tx(tx):
    result = tx.run(TX_STR)
    print([f'result.{fn}' for fn in dir(result) if not fn.startswith('_')])
    record_dict = result.single()
    
    return record_dict
session = driver.session()
session.read_transaction(display_nodes_tx)

In [2]:

import os
from ctypes import WinDLL
import glob
import sys

libs_path = r'D:\Documents\GitHub\job-hunting\jh_env\Lib\site-packages\scipy\.libs'
try:
    owd = os.getcwd()
    os.chdir(libs_path)
    for filename in glob.glob(os.path.join(libs_path, '*dll')):
        file_path = os.path.abspath(filename)
        print(file_path)
        try:
            WinDLL(file_path)
        except Exception as e:
            message = str(e).strip()
            print(message)
#             os.remove(file_path)
finally:
    os.chdir(owd)

D:\Documents\GitHub\job-hunting\jh_env\Lib\site-packages\scipy\.libs\libansari.R6EA3HQP5KZ6TAXU4Y4ZVTRPT7UVA53Z.gfortran-win_amd64.dll
D:\Documents\GitHub\job-hunting\jh_env\Lib\site-packages\scipy\.libs\libansari.VQIMK74KXZROCK2N4NPKQNPUWDMM7QOK.gfortran-win_amd64.dll
D:\Documents\GitHub\job-hunting\jh_env\Lib\site-packages\scipy\.libs\libbanded5x.R6SL6MWEG32T6CYYAC2G6L7N2F22MDU6.gfortran-win_amd64.dll
D:\Documents\GitHub\job-hunting\jh_env\Lib\site-packages\scipy\.libs\libbispeu.7AH3PCQ2E2NGLC3AQD7FFAH73KGJTZCJ.gfortran-win_amd64.dll
D:\Documents\GitHub\job-hunting\jh_env\Lib\site-packages\scipy\.libs\libbispeu.W5IKIQ633I7FHCSXA72OUIBHYPZECT25.gfortran-win_amd64.dll
D:\Documents\GitHub\job-hunting\jh_env\Lib\site-packages\scipy\.libs\libblkdta00.TX7UXLIMOXUZRMP6FM3A3C7KWZUR4NCQ.gfortran-win_amd64.dll
D:\Documents\GitHub\job-hunting\jh_env\Lib\site-packages\scipy\.libs\libchkder.G7WSOGIYYQO3UWFVEZ3PPXCXR53ADVPA.gfortran-win_amd64.dll
D:\Documents\GitHub\job-hunting\jh_env\Lib\site-pac